In [ ]:
%pip install opencv-python opencv-python-headless ultralytics easyocr



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [161]:
from ultralytics import YOLO
import easyocr
import os
import cv2
import numpy as np
import time


In [197]:
yolo_model = YOLO('yolov5s.pt')  # Pre-trained model
reader = easyocr.Reader(['en'])


yolo_weights = '/Users/mohan/Desktop/NumberplateRecognition/yolov4.weights'
yolo_cfg = '/Users/mohan/Desktop/NumberplateRecognition/yolov4.cfg'
yolo_names = '/Users/mohan/Desktop/NumberplateRecognition/coco.names'

PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [198]:
net = cv2.dnn.readNet(yolo_weights, yolo_cfg)

with open(yolo_names, 'r') as f:
    classes = f.read().strip().split('\n')


layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]


In [207]:
preprocessed_dir = "preprocessed_images"
car_regions_dir = "car_regions"
os.makedirs(preprocessed_dir, exist_ok=True)
os.makedirs(car_regions_dir, exist_ok=True)



In [208]:
def preprocess_image(image, frame_num):
    """Preprocess the image for better OCR."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    filename = os.path.join(preprocessed_dir, f"frame_{frame_num}.jpg")
    cv2.imwrite(filename, gray)
    return gray


In [209]:

def detect_number_plate(image):
    """Detect number plate text using EasyOCR and handle multiple detections."""
    results = reader.readtext(image)

    # Initialize variables to store the best result
    best_text = ""
    best_confidence = 0.0

    # Loop through detected results to filter the best one
    for res in results:
        text, confidence = res[1], res[2]
        
        # Filter out results with low confidence or very short text
        if confidence > 0.5 and len(text) > 2:
            if confidence > best_confidence:
                best_text = text
                best_confidence = confidence

    return best_text if best_text else "No valid text detected"

In [210]:
def load_yolo_model(weights, cfg, names):
    """Load YOLOv4 model using OpenCV's DNN module."""
    net = cv2.dnn.readNet(weights, cfg)
    with open(names, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    return net, classes

In [211]:
def detect_car_regions_yolo(frame, net, classes):
    """Detect car regions in the frame using YOLOv4."""
    height, width, _ = frame.shape

    # Prepare the image for YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(net.getUnconnectedOutLayersNames())

    car_regions = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.5 and classes[class_id] == "car":  # Class "car" for vehicle detection
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                car_regions.append((x, y, w, h))

    return car_regions

In [212]:
def process_input(car_region, frame_num):
    """Process the car region and extract text."""
    try:
        # Preprocess the car region and extract text
        preprocessed_image = preprocess_image(car_region, frame_num)
        text = detect_number_plate(preprocessed_image)

        if text and text != "No valid text detected":
            print(f"Frame {frame_num}: Detected Text: {text}")
            return text
        else:
            # Print only once per frame if no valid text is detected
            if frame_num % 10 == 0:  # Print every 10 frames or as desired
                print(f"Frame {frame_num}: No valid text detected.")
            return "No valid text detected"

    except Exception as e:
        print(f"An error occurred while processing frame {frame_num}: {e}")
        return "Error in text detection"


In [213]:
def process_video(video_path, net, classes):
    cap = cv2.VideoCapture(video_path)
    frame_num = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_num += 1

        # Process every 90th frame (approx. every 3 seconds at 30 FPS)
        if frame_num % 90 == 0:
            print(f"Processing frame {frame_num}...")

            # Detect car regions using YOLO
            car_regions = detect_car_regions_yolo(frame, net, classes)

            frame_height, frame_width = frame.shape[:2]

            for (x, y, w, h) in car_regions:
                # Validate region coordinates
                if x < 0 or y < 0 or x + w > frame_width or y + h > frame_height:
                    print(f"Skipping invalid region: x={x}, y={y}, w={w}, h={h}")
                    continue  # Skip invalid regions

                # Extract the car region
                car_region = frame[y:y + h, x:x + w]

                # Check if the car region is valid (non-empty)
                if car_region.size == 0:
                    print(f"Skipping empty region: x={x}, y={y}, w={w}, h={h}")
                    continue

                # Save the car region
                filename = os.path.join(car_regions_dir, f"frame_{frame_num}_{x}_{y}.jpg")
                cv2.imwrite(filename, car_region)

                # Process the car region to extract number plate text
                plate_text = process_input(car_region, frame_num)
                print(f"Detected Text: {plate_text}")

                # Display the detected text on the frame
                cv2.putText(frame, plate_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        # Display the frame with detected car regions and text
        cv2.imshow("Video Feed", frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [214]:
if __name__ == "__main__":
    # Set your YOLOv4 paths
    yolo_weights = '/Users/mohan/Desktop/NumberplateRecognition/yolov4.weights'
    yolo_cfg = '/Users/mohan/Desktop/NumberplateRecognition/yolov4.cfg'
    yolo_names = '/Users/mohan/Desktop/NumberplateRecognition/coco.names'

    # Load the YOLOv4 model
    yolo_net, yolo_classes = load_yolo_model(yolo_weights, yolo_cfg, yolo_names)

    # Path to your video file
    video_path = '/Users/mohan/Downloads/Automatic Number Plate Recognition (ANPR) _ Vehicle Number Plate Recognition (1).mp4'  # Replace with your video file path or 0 for live camera

    # Process the video
    process_video(video_path, yolo_net, yolo_classes)

Processing frame 90...
Frame 90: No valid text detected.
Detected Text: No valid text detected
Frame 90: No valid text detected.
Detected Text: No valid text detected
Frame 90: No valid text detected.
Detected Text: No valid text detected
Frame 90: No valid text detected.
Detected Text: No valid text detected
Frame 90: No valid text detected.
Detected Text: No valid text detected
Frame 90: No valid text detected.
Detected Text: No valid text detected
Frame 90: No valid text detected.
Detected Text: No valid text detected
Frame 90: No valid text detected.
Detected Text: No valid text detected
Frame 90: No valid text detected.
Detected Text: No valid text detected
Frame 90: No valid text detected.
Detected Text: No valid text detected
Frame 90: No valid text detected.
Detected Text: No valid text detected
Frame 90: No valid text detected.
Detected Text: No valid text detected
Frame 90: No valid text detected.
Detected Text: No valid text detected
Frame 90: No valid text detected.
Detecte